In [3]:
# -------------------------------
# 0. 라이브러리 및 환경 설정
# -------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from scipy.stats import zscore, pointbiserialr
from sklearn.preprocessing import PowerTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 그래프 한글폰트(선택) ─ macOS 예시
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

In [8]:
kospi = pd.read_csv("../data/raw/코스피_상장폐지.csv")
kosdaq = pd.read_csv("../data/raw/코스닥_상장폐지.csv")

In [9]:
kospi['폐지사유'].unique().tolist()

['신청에 의한 상장폐지',
 '감사의견 의견거절 2년 계속',
 '지주회사(최대주주등)의 완전자회사화 등',
 '해산 사유 발생',
 '감사의견 부적정',
 '존속기간 만료',
 '감사의견 의견거절',
 '기업의 계속성, 경영의 투명성 및 기타 공익과 투자자 보호 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
 '자본전액잠식',
 '시가총액 미달',
 '매출액 미달(50억원 미만) 2년 계속',
 '공시서류 미제출(사업보고서) 후 10일이내 미제출',
 '보통주 주식분포요건 미충족 2년 계속',
 '지주회사의 완전자회사화(지주회사 신규상장)',
 'SPAC 상장예비심사청구서 미제출 등',
 '보통주 주가수준미달',
 '자본잠식(자본금의 50/100이상 잠식) 2년 계속',
 '최종부도발생']

In [16]:
kosdaq['폐지사유'].unique().tolist()

['상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소',
 '피흡수합병(스팩소멸합병)',
 '타법인의 완전자회사로 편입',
 '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정',
 '감사의견 거절(감사범위 제한)',
 "상장폐지 신청('23.06.28)",
 '유가증권시장 상장',
 '감사의견 거절(감사범위 제한 및 계속기업가정 불확실성)',
 '감사의견 거절(감사범위 제한 및 계속기업 존속능력 불확실성)',
 '피흡수합병',
 '기업의 계속성 및 경영의 투명성 등을 종합적을 고려하여 상장폐지기준에 해당한다고 결정',
 "상장폐지 신청('22.08.31)",
 '상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동사유 미해소',
 "상장폐지 신청('22.03.30)",
 '감사의견 거절',
 '감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)',
 "상장폐지 신청('22.04.14)",
 '발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 거래정지',
 '해산사유 발생(파산선고)',
 '상장예비심사청구서 미제출',
 '감사의견 한정(감사범위 제한)',
 '법정제출기한까지 사업보고서를 제출하지 아니한 후, 법정제출기한의 다음날부터 10일이내에 사업보고서를 제출하지 아니함',
 '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당',
 '최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미제출',
 '감사의견거절(감사범위 제한)',
 '감사의견거절(감사범위제한)',
 '감사의견거절(감사범위제한 및 계속기업 존속 불확실)',
 '제28조제1항제9호에 따라 관리종목 지정 후 공시규정 제19조제1항에 따른 사업보고서 법정제출기한 내 미제출, 최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미

In [23]:
c_ratio = pd.read_csv("../data/raw/연결 재무비율(IFRS).csv")
n_ratio = pd.read_csv("../data/raw/재무비율(IFRS).csv")
c_bs = pd.read_csv("../data/raw/연결 재무제표(IFRS).csv")
n_bs = pd.read_csv("../data/raw/재무제표(IFRS).csv")
delisting = pd.concat([kospi, kosdaq])
df = pd.DataFrame()

In [ ]:
print("--- '회사명' 기준 비교 결과 ---")
# kospi 데이터프레임에서 유니크한 회사명 추출
kospi_companies = delisting['회사명'].unique().tolist()
print(f"상장폐지 데이터프레임의 유니크한 회사명 목록 (총 {len(kospi_companies)}개): {kospi_companies}\n")

# c_bs 데이터프레임에서 유니크한 회사명 추출
c_bs_companies = c_bs['회사명'].unique().tolist()
print(f"c_bs 데이터프레임의 유니크한 회사명 목록 (총 {len(c_bs_companies)}개): {c_bs_companies}\n")

# 리스트를 set으로 변환하여 비교 효율성 높이기
set_kospi_comp = set(kospi_companies)
set_c_bs_comp = set(c_bs_companies)

# 1. KOSPI에만 있는 회사명
unique_to_kospi_comp = set_kospi_comp - set_c_bs_comp
print(f"상장폐지에만 있는 회사명 (총 {len(unique_to_kospi_comp)}개): {unique_to_kospi_comp}")

# 2. c_bs에만 있는 회사명
unique_to_c_bs_comp = set_c_bs_comp - set_kospi_comp
print(f"c_bs에만 있는 회사명 (총 {len(unique_to_c_bs_comp)}개): {unique_to_c_bs_comp}")

# 3. 두 데이터프레임 모두에 있는 회사명 (공통)
common_companies_comp = set_kospi_comp.intersection(set_c_bs_comp)
print(f"두 데이터프레임 모두에 있는 회사명 (공통, 총 {len(common_companies_comp)}개): {common_companies_comp}")

# 4. 전체 유니크한 회사명 (합집합)
all_unique_companies_comp = set_kospi_comp.union(set_c_bs_comp)
print(f"두 데이터프레임의 모든 유니크한 회사명 (합집합, 총 {len(all_unique_companies_comp)}개): {all_unique_companies_comp}")

# 5. 두 데이터프레임 중 한쪽에만 있는 회사명 (대칭 차집합)
symmetric_difference_comp = set_kospi_comp.symmetric_difference(set_c_bs_comp)
print(f"두 데이터프레임 중 한쪽에만 있는 회사명 (대칭 차집합, 총 {len(symmetric_difference_comp)}개): {symmetric_difference_comp}\n")

--- '회사명' 기준 비교 결과 ---
KOSPI 데이터프레임의 유니크한 회사명 목록 (총 482개): ['바다로19호', '에코바이브', '우리종금', '메리츠증권', '메리츠화재', '베트남개발1', '롯데푸드', '하이골드3호', '엔에스쇼핑', '폴루스바이오팜', '부산도시가스', 'SBS미디어홀딩스', '하이골드12호', '코오롱머티리얼', '우리금융캐피탈', '동성화학', '코리아오토글라스', '고려개발', '한국제지', '하나니켈2호', '하나니켈1호', '신한', '동북아12호선박투자', '웅진에너지', '동북아13호선박투자', '하이골드8호', '한화갤러리아타임월드', '두산건설', '오렌지라이프', '사조해표', '알보젠코리아', '트러스제7호', '우리은행', '대덕GDS', '한국유리공업', '광주은행', '성지건설', '도레이케미칼', '중국원양자원', '대성합동지주', '아시아퍼시픽15호', 'KB캐피탈', 'KB손해보험', '보루네오가구', '아시아퍼시픽14호', '아시아퍼시픽13호', '아시아퍼시픽12호', '아시아퍼시픽11호', '아시아10호', '동북아11호선박투자', '동북아14호선박투자', '동북아10호선박투자', '코리아03호', '코리아02호', '코리아01호', '코리아04호', '넥솔론', '한솔아트원제지', '한진해운', '미래에셋증권', 'LG생명과학', '하이골드2호', '신성이엔지', '신성에프에이', '현대페인트', 'KB증권', '태림페이퍼', '바다로3호', '경남에너지', '티이씨코', '플랜텍', '한화화인케미칼', '삼양제넥스', '평산차업', '코크렙15호', '이코리아리츠', '삼성물산', 'SK', '현대하이스코', '경남은행', '삼환기업', '경남기업', '신일건업', '아시아5호', '아시아6호', '아시아7호', '케이비부국위탁리츠', '로케트전기', '아시아8호', 'NH농협증권', '아시아4호', '코크렙8호', '유니온스틸', '아시아9호', 'SBI모기지', '삼양엔텍', '아시

In [ ]:
print("--- '종목코드' 기준 비교 결과 ---")
# kospi 데이터프레임에서 유니크한 종목코드 추출
kospi_stock_codes = delisting['종목코드'].unique().tolist()
print(f"상장폐지 데이터프레임의 유니크한 종목코드 목록 (총 {len(kospi_stock_codes)}개): {kospi_stock_codes}\n")

# c_bs 데이터프레임에서 유니크한 종목코드 추출
c_bs_stock_codes = c_bs['거래소코드'].unique().tolist()
print(f"c_bs 데이터프레임의 유니크한 종목코드 목록 (총 {len(c_bs_stock_codes)}개): {c_bs_stock_codes}\n")

# 리스트를 set으로 변환하여 비교 효율성 높이기
set_kospi_codes = set(kospi_stock_codes)
set_c_bs_codes = set(c_bs_stock_codes)

# 1. KOSPI에만 있는 종목코드
unique_to_kospi_codes = set_kospi_codes - set_c_bs_codes
print(f"상장폐지에만 있는 종목코드 (총 {len(unique_to_kospi_codes)}개): {unique_to_kospi_codes}")

# 2. c_bs에만 있는 종목코드
unique_to_c_bs_codes = set_c_bs_codes - set_kospi_codes
print(f"c_bs에만 있는 종목코드 (총 {len(unique_to_c_bs_codes)}개): {unique_to_c_bs_codes}")

# 3. 두 데이터프레임 모두에 있는 종목코드 (공통)
common_stock_codes = set_kospi_codes.intersection(set_c_bs_codes)
print(f"두 데이터프레임 모두에 있는 종목코드 (공통, 총 {len(common_stock_codes)}개): {common_stock_codes}")

# 4. 전체 유니크한 종목코드 (합집합)
all_unique_stock_codes = set_kospi_codes.union(set_c_bs_codes)
print(f"두 데이터프레임의 모든 유니크한 종목코드 (합집합, 총 {len(all_unique_stock_codes)}개): {all_unique_stock_codes}")

# 5. 두 데이터프레임 중 한쪽에만 있는 종목코드 (대칭 차집합)
symmetric_difference_codes = set_kospi_codes.symmetric_difference(set_c_bs_codes)
print(f"두 데이터프레임 중 한쪽에만 있는 종목코드 (대칭 차집합, 총 {len(symmetric_difference_codes)}개): {symmetric_difference_codes}")


--- '종목코드' 기준 비교 결과 ---
KOSPI 데이터프레임의 유니크한 종목코드 목록 (총 482개): [155900, 15540, 10050, 8560, 60, 96300, 2270, 153360, 138250, 7630, 15350, 101060, 172580, 144620, 33660, 5190, 152330, 4200, 2300, 99350, 99340, 5450, 83370, 103130, 83380, 159650, 27390, 11160, 79440, 79660, 2250, 140890, 30, 4130, 2000, 192530, 5980, 8000, 900050, 5620, 83620, 21960, 2550, 4740, 83610, 83600, 83590, 83580, 83570, 83360, 83390, 83350, 90990, 90980, 90970, 91000, 110570, 7190, 117930, 37620, 68870, 139200, 104110, 104120, 11720, 3450, 19300, 92630, 8020, 8900, 51310, 25850, 3940, 950010, 121550, 138440, 830, 3600, 10520, 192520, 360, 800, 14350, 82240, 82250, 82260, 149130, 420, 81930, 16420, 81210, 90540, 3640, 81940, 950100, 8720, 81200, 81190, 53000, 119250, 1300, 80180, 101990, 5900, 2530, 80980, 10460, 67250, 5560, 80970, 80960, 950070, 8080, 80030, 6350, 12410, 3190, 4940, 9280, 6440, 93400, 93410, 99210, 16570, 7480, 9380, 4010, 14300, 64420, 122750, 15110, 26870, 122290, 7200, 16560, 121910, 900030, 

In [30]:
c_bs.columns.tolist()

['회사명',
 '거래소코드',
 '회계년도',
 '[A100000000]자산(*)(IFRS연결)(천원)',
 '[A600000000]자본(*)(IFRS연결)(천원)',
 '[A600010000]* 수권주식수(IFRS연결)(주)',
 '[A600010100]* 주당 액면가액(IFRS연결)(원)',
 '[A600010200]* 발행한 주식총수(*)(IFRS연결)(천원)',
 '[A600010300]   보통주(IFRS연결)(주)',
 '[A600010400]   우선주(IFRS연결)(주)',
 '[A610000000]   지배기업주주지분(*)(IFRS연결)(천원)',
 '[A611000000]      자본금(*)(IFRS연결)(천원)',
 '[A611010000]         보통주자본금(IFRS연결)(천원)',
 '[A611010100]         우선주자본금(IFRS연결)(천원)',
 '[A611050000]      신종자본증권(IFRS연결)(천원)',
 '[A612000000]      자본잉여금(*)(IFRS연결)(천원)',
 '[A612010000]         주식발행초과금(IFRS연결)(천원)',
 '[A612010100]         배당건설이자(IFRS연결)(천원)',
 '[A612010200]         주식매입선택권(IFRS연결)(천원)',
 '[A612010300]         감자차익(IFRS연결)(천원)',
 '[A612010400]         보험차익(IFRS연결)(천원)',
 '[A612010500]         합병차익(IFRS연결)(천원)',
 '[A612010600]         자산수증이익(IFRS연결)(천원)',
 '[A612010700]         채무면제이익(IFRS연결)(천원)',
 '[A612010800]         자기주식처분이익(IFRS연결)(천원)',
 '[A612010900]         전환권대가(IFRS연결)(천원)',
 '[A612011000]         신주인수권대가

In [29]:
c_ratio.columns.tolist()

['회사명',
 '거래소코드',
 '회계년도',
 '총자본증가율(IFRS연결)',
 '자기자본증가율(IFRS연결)',
 '매출액증가율(IFRS연결)',
 '정상영업이익증가율(IFRS연결)',
 '순이익증가율(IFRS연결)',
 '총포괄이익증가율(IFRS연결)',
 '종업원1인당 부가가치증가율(IFRS연결)',
 '종업원수증가율(IFRS연결)',
 '종업원1인당 매출액증가율(IFRS연결)',
 '종업원1인당 인건비증가율(IFRS연결)',
 '매출액정상영업이익률(IFRS연결)',
 '매출액순이익률(IFRS연결)',
 '총자본사업이익률(IFRS연결)',
 '총자본정상영업이익률(IFRS연결)',
 '총자본순이익률(IFRS연결)',
 '자기자본정상영업이익률(IFRS연결)',
 '자기자본순이익률(IFRS연결)',
 '자본금정상영업이익률(IFRS연결)',
 '자본금순이익률(IFRS연결)',
 '영업비용 대 영업수익비율(IFRS연결)',
 '수지비율(관계기업투자손익 제외)(IFRS연결)',
 '세금과공과 대 총비용비율(IFRS연결)',
 '유보율(IFRS연결)',
 '사내유보율(IFRS연결)',
 '사내유보 대 자기자본비율(IFRS연결)',
 '적립금비율(재정비율)(IFRS연결)',
 '평균배당률(IFRS연결)',
 '자기자본배당률(IFRS연결)',
 '배당성향(IFRS연결)',
 '1주당매출액(IFRS연결)(원)',
 '1주당순이익(IFRS연결)(원)',
 '1주당 CASH FLOW(IFRS연결)(원)',
 '1주당순자산(IFRS연결)(원)',
 '1주당정상영업이익(IFRS연결)(원)',
 '자기자본구성비율(IFRS연결)',
 '타인자본구성비율(IFRS연결)',
 '부채비율(IFRS연결)',
 '차입금의존도(IFRS연결)',
 '차입금비율(IFRS연결)',
 '유보액대비율(IFRS연결)',
 'CASH FLOW 대 총자본비율(IFRS연결)',
 '총자본회전률(IFRS연결)',
 '자기자본회전률(IFRS연결)',
 '자본금회전률(IFRS연결)',
 '타인자본회전률(IFRS연

In [ ]:
# -------------------------------
# 1. 데이터 불러오기
# -------------------------------
### TODO: 실제 파일 경로를 지정하세요
df = pd.read_csv("../data/raw/연결 재무제표")

# -------------------------------
# 2. 주요 변수 지정
# -------------------------------
### TODO: 컬럼명을 실제 데이터에 맞춰 수정
id_col      = 'ticker'
target_col  = 'label'          # 부실=1, 정상=0
num_cols    = [                # 수치형 재무/파생 변수 목록
    'total_assets', 'total_liab',
    'sales', 'net_income',
    'current_ratio', 'debt_ratio',
    'altman_k2',  # …
]
cat_cols = []                  # 범주형 변수 있다면 추가

In [ ]:
# 2-1. 정보 요약
display(df.info())
display(df.head())

# 2-2. 중복 확인 (동일 기업·연도 중복행 탐지)
dup_mask = df.duplicated(subset=[id_col, 'year'])
print(f"중복 행 수: {dup_mask.sum()}")
if dup_mask.any():
    df = df[~dup_mask]  # 필요 시 제거

In [ ]:
# 3-1. 변수별 결측 비율
null_ratio = df[num_cols + [target_col]].isna().mean().sort_values(ascending=False)
display(null_ratio.head(10))

# 3-2. 히트맵 시각화
plt.figure(figsize=(10,4))
msno.matrix(df[num_cols + [target_col]].sample(min(500, len(df))))
plt.show()

In [ ]:
# 4-1. 기술통계
desc = df[num_cols].describe().T
display(desc)

# 4-2. z-score 기반 이상치 마스크
z_scores = df[num_cols].apply(zscore)
outlier_mask = (np.abs(z_scores) > 3).any(axis=1)
print(f"z>|3| 이상치 행 비율: {outlier_mask.mean():.2%}")

# 4-3. 상자그림(샘플 6개 변수)
plot_cols = num_cols[:6]
sns.boxplot(data=df[plot_cols], orient='h'); plt.title("Boxplot (sample)"); plt.show()

In [ ]:
# 5-1. 오른쪽 꼬리가 긴 양수 변수에 log1p 적용 예시
log_cols = ['sales', 'total_assets']    # TODO: 필요 변수 지정
df[log_cols] = df[log_cols].apply(np.log1p)

# 5-2. Yeo-Johnson 변환 예시
yj_cols = ['current_ratio']             # TODO: 필요 변수 지정
pt = PowerTransformer(method='yeo-johnson')
df[yj_cols] = pt.fit_transform(df[yj_cols])

In [ ]:
# 6-1. 피어슨 상관행렬
corr = df[num_cols].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', center=0, annot=False)
plt.title("Correlation Matrix"); plt.show()

# 6-2. VIF 계산
X = df[num_cols].drop(columns=[], errors='ignore')  # 필요 시 제외 변수 지정
vif_vals = pd.Series(
    [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
    index=X.columns, name='VIF'
)
display(vif_vals.sort_values(ascending=False).head(10))

In [ ]:
# 7-1. 포인트-바이시리얼 상관계수
pb_dict = {
    col: pointbiserialr(df[col].fillna(0), df[target_col])[0]
    for col in num_cols
}
pb_series = pd.Series(pb_dict, name='point_biserial')
display(pb_series.sort_values(key=np.abs, ascending=False).head(10))

# 7-2. 변수별 상자그림 (부실 vs 정상)
plt.figure(figsize=(12,4))
for i, c in enumerate(num_cols[:3], 1):    # 샘플 3개 변수
    plt.subplot(1,3,i)
    sns.boxplot(x=target_col, y=c, data=df)
    plt.title(c)
plt.tight_layout(); plt.show()

In [ ]:
class_counts = df[target_col].value_counts().sort_index()
print(class_counts)
print("부실(1) 비율:", class_counts.get(1,0) / class_counts.sum())

# 파이차트
plt.figure(figsize=(4,4))
class_counts.plot.pie(autopct='%1.1f%%', labels=['정상(0)','부실(1)'])
plt.ylabel('')
plt.title("Class Distribution"); plt.show()